📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load an Image](#toc2_)    
- [Color Space Conversion](#toc3_)    
  - [RGB - Grayscale](#toc3_1_)    
    - [Manual](#toc3_1_1_)    
    - [Using OpenCV](#toc3_1_2_)    
    - [Using PIL](#toc3_1_3_)    
    - [Using Scikit-Image](#toc3_1_4_)    
  - [RGB - BGR](#toc3_2_)    
    - [Manual](#toc3_2_1_)    
    - [Using OpenCV](#toc3_2_2_)    
  - [RGB - YUV](#toc3_3_)    
    - [Manual](#toc3_3_1_)    
    - [Using OpenCV](#toc3_3_2_)    
  - [RGB - YCrCb](#toc3_4_)    
    - [Manual](#toc3_4_1_)    
    - [Using OpenCV](#toc3_4_2_)    
  - [RGB - HSV](#toc3_5_)    
    - [Using OpenCV](#toc3_5_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from numpy.typing import NDArray
from PIL import Image

# <a id='toc2_'></a>[Load an Image](#toc0_)

In [ ]:
im_bgr = cv2.imread(filename="../../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif")
im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
im_rgb_2 = Image.fromarray(im_rgb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["Original", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_rgb, im_rgb[:, :, 0], im_rgb[:, :, 1], im_rgb[:, :, 2]]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if len(img.shape) == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

# <a id='toc3_'></a>[Color Space Conversion](#toc0_)

📝 **Docs**:

- `cv2.cvtColor`: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14)
- Color Space Conversions [`cv2`]: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html)
- Modes [`PIL`]: [pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)
- Colormap reference [`matplotlib`]: [matplotlib.org/stable/gallery/color/colormap_reference.html](https://matplotlib.org/stable/gallery/color/colormap_reference.html)
- ITU-R Recommendation BT.601-7 [an international technical standard]: [itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf](http://itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf)

## <a id='toc3_1_'></a>[RGB - Grayscale](#toc0_)

### <a id='toc3_1_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_GS = np.array(
    [
        [0.299, 0.587, 0.114],
    ],
    dtype=np.float64,
)

In [ ]:
def rgb_to_gs(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_GS.T).reshape(image.shape[:2])
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_rgb_to_gs_1 = rgb_to_gs(im_rgb)

# plot
plt.imshow(im_rgb_to_gs_1, cmap="gray")
plt.title("GrayScale")
plt.axis("off")
plt.show()

### <a id='toc3_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_rgb_to_gs_2 = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2GRAY)

# plot
plt.imshow(im_rgb_to_gs_2, cmap="gray")
plt.title("GrayScale")
plt.axis("off")
plt.show()

### <a id='toc3_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
im_rgb_to_gs_3 = Image.fromarray(im_rgb).convert("L")

# plot
plt.imshow(im_rgb_to_gs_3, cmap="gray")
plt.title("GrayScale")
plt.axis("off")
plt.show()

### <a id='toc3_1_4_'></a>[Using Scikit-Image](#toc0_)


In [ ]:
im_rgb_to_gs_4 = ski.color.rgb2gray(im_rgb)

# plot
plt.imshow(im_rgb_to_gs_4, cmap="gray")
plt.title("GrayScale")
plt.axis("off")
plt.show()

## <a id='toc3_2_'></a>[RGB - BGR](#toc0_)

### <a id='toc3_2_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_BGR = np.array(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=np.float64,
)

BGR_TO_RGB = np.array(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=np.float64,
)

In [ ]:
def rgb_to_bgr(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_BGR.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def bgr_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ BGR_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_rgb_to_bgr = rgb_to_bgr(im_rgb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_rgb_to_bgr] + [im_rgb_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_2_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_rgb_to_bgr = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2BGR)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Blue Channel", "Green Channel", "Red Channel"]
images = [im_rgb_to_bgr] + [im_rgb_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_3_'></a>[RGB - YUV](#toc0_)

### <a id='toc3_3_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_YUV = np.array(
    [
        [0.299, 0.587, 0.114],
        [-0.14713, -0.28886, 0.436],
        [0.615, -0.51499, -0.10001],
    ],
    dtype=np.float64,
)

YUV_TO_RGB = np.array(
    [
        [1.0, 0.0, 1.13983],
        [1.0, -0.39465, -0.58060],
        [1.0, 2.03211, 0.0],
    ],
    dtype=np.float64,
)

YUV_OFFSET = np.array([0, 128, 128])

In [ ]:
def rgb_to_yuv(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YUV.T + YUV_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def yuv_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YUV_OFFSET) @ YUV_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_rgb_to_yuv = rgb_to_yuv(im_rgb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_rgb_to_yuv] + [im_rgb_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_3_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_rgb_to_yuv = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2YUV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_rgb_to_yuv] + [im_rgb_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_4_'></a>[RGB - YCrCb](#toc0_)

### <a id='toc3_4_1_'></a>[Manual](#toc0_)


In [ ]:
np.array(
    [
        [65.481, 128.553, 24.966],
        [-37.797, -74.203, 112.0],
        [112.0, -93.786, -18.214],
    ],
    dtype=np.float64,
) / 255.0

In [ ]:
RGB_TO_YCRCB = (
    np.array(
        [
            [65.481, 128.553, 24.966],
            [112.0, -93.786, -18.214],
            [-37.797, -74.203, 112.0],
        ],
        dtype=np.float64,
    )
    / 255.0
)

YCRCB_TO_RGB = np.array(
    [
        [1.164, 0.000, 1.596],
        [1.164, 2.017, 0.000],
        [1.164, -0.392, -0.813],
    ],
    dtype=np.float64,
)

YCRCB_OFFSET = np.array([16, 128, 128])

In [ ]:
def rgb_to_ycrcb(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YCRCB.T + YCRCB_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def ycrcb_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YCRCB_OFFSET) @ YCRCB_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_rgb_to_ycrcb = rgb_to_ycrcb(im_rgb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_rgb_to_ycrcb] + [im_rgb_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_4_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_rgb_to_ycrcb = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2YCrCb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_rgb_to_ycrcb] + [im_rgb_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_5_'></a>[RGB - HSV](#toc0_)

### <a id='toc3_5_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_rgb_to_hsv = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2HSV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["HSV Image", "Hue Channel", "Saturation Channel", "Value Channel"]
images = [im_rgb_to_hsv] + [im_rgb_to_hsv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()